In [2]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
base_path= "drive/My Drive/AI_LAWYER/brat-project-final/essay" # add essay number and ".txt" or ".ann"
total_essays=402

In [6]:
def getPathEnd(val):
    if val>=100:
        return str(val)
    elif val>=10:
        return "0"+str(val)
    return "00"+str(val)

def getTrainTest(val):
    return val.split(";")[1][1:-1] == "TRAIN"

In [7]:
df = pd.read_csv("drive/My Drive/AI_LAWYER/train-test-split.csv")
# df

In [8]:
def parseSentence(para , not_words):
    _lengths = [ len(_w) for _w in not_words ]
    _num_stop_words = len(not_words)
    _pos = 0
    _para_length  = len(para)
    _i=0
    _sentences = []
    while _i<_para_length:
        _j = 0 
        _k=0
        while _j<_num_stop_words:
            if _para_length > _i+_lengths[_j]:
                if para[_i : _i+_lengths[_j]]  == not_words[_j]:
                    _i+= _lengths[_j]
                    _k=1
                    break
            _j+=1
        # print(_i)
        if para[_i] == '.':
            _sentences.append(para[_pos:_i+1])
            _pos=_i+1
        if _k==0:
            _i+=1
    if _pos<_para_length:
        _sentences.append(para[_pos:_para_length])
    return _sentences

In [47]:
test_data_classifier1 = []
test_data_classifier2 = []
train_data_classifier1 = []
train_data_classifier2 = []

In [ ]:
k=0
for i in range(total_essays):
    total_path = base_path + getPathEnd(i+1)
    ann_file = open(total_path + ".ann", 'r')
    sentence = ann_file.read().split("\n")[-1]
    ann_file.close()
    if sentence!='':
        k+=1
print(k)
# all '.ann' files end with a blank line 

0


In [9]:
not_words = ['Mr.', 'Mrs.', 'Ms.', 'i.e.','etc.',  'e.g.','Dr.', 'Prof.', 'Rev.', 'Capt.', 'Lt.-Col.', 'Col.']

In [48]:
for essay_num in range(total_essays):
    total_path = base_path + getPathEnd(essay_num+1)
    ann_file = open(total_path + ".ann", 'r')
    sentences = ann_file.read().split("\n")[:-1]
    ann_file.close()
    _train_ = getTrainTest(df['ID;"SET"'][essay_num])
    txt_file = open(total_path + ".txt", 'r')
    paras = txt_file.read().split("\n")
    txt_file.close()

    unannotated_sentences = []
    for temp_paras in paras: 
        unannotated_sentences += parseSentence(temp_paras , not_words) 
        unannotated_sentences.append("\n")
    unannotated_sentences.pop(-1)
    unannotated_sentences_length = [len(s_) for s_ in unannotated_sentences]
    num_unannotated_sentences = len(unannotated_sentences)

    for i in range(len(unannotated_sentences_length)-1):
        unannotated_sentences_length[i+1]+= unannotated_sentences_length[i]

    for i in range(len(unannotated_sentences)):
        unannotated_sentences[i] = {"sentence":unannotated_sentences[i] , "has_argument":False , "start_boundary":-1, "end_boundary":-1,"argument":'-', 'type':'-'}
    
    # print(unannotated_sentences)
    for sentence in sentences:
        temp_splits = sentence.split("\t")
    
        if temp_splits[0][0]=='T':
            temp_splits2 = temp_splits[1].split()
            temp_l = 0
            temp_r = num_unannotated_sentences-1
            if int(temp_splits2[1])<unannotated_sentences_length[0]:
                unannotated_sentences[0]["argument"] =  temp_splits[2]
                unannotated_sentences[0]["has_argument"] =True
                unannotated_sentences[0]["start_boundary"] = temp_splits2[1]
                unannotated_sentences[0]["end_boundary"] = temp_splits2[2]
                unannotated_sentences[0]["type"] = temp_splits2[0]
            j_ = 1
            while j_ < num_unannotated_sentences:
                if unannotated_sentences_length[j_-1]<=int(temp_splits2[1]) and unannotated_sentences_length[j_]>=int(temp_splits2[2]):
                    unannotated_sentences[j_]["argument"] =  temp_splits[2]
                    unannotated_sentences[j_]["has_argument"] =True
                    unannotated_sentences[j_]["start_boundary"] = int(temp_splits2[1]) - unannotated_sentences_length[j_-1]
                    unannotated_sentences[j_]["end_boundary"] = int(temp_splits2[2]) - unannotated_sentences_length[j_-1]
                    unannotated_sentences[j_]["type"] = temp_splits2[0]
                    break
                j_+=1   

    for sen in unannotated_sentences:
        if len(sen["sentence"])>2:
            if _train_:
                train_data_classifier1.append({"essay no.":essay_num+1,"sentence":sen["sentence"] , "has_argument":sen["has_argument"], "type":sen["type"]})
                if sen["has_argument"]:
                    train_data_classifier2.append({"essay no.":essay_num+1 , "sentence":sen["sentence"] , "argument":sen["argument"], "type":sen["type"],"start":sen["start_boundary"], "end":sen["end_boundary"]})    
            else:
                test_data_classifier1.append({"essay no.":essay_num+1,"sentence":sen["sentence"] , "has_argument":sen["has_argument"], "type":sen["type"]})
                if sen["has_argument"]:
                    test_data_classifier2.append({"essay no.":essay_num+1 , "sentence":sen["sentence"] , "argument":sen["argument"], "type":sen["type"],"start":sen["start_boundary"], "end":sen["end_boundary"]})    
            
    if (essay_num+1)%50==0:
        print(essay_num , " DOne")

49  DOne
99  DOne
149  DOne
199  DOne
249  DOne
299  DOne
349  DOne
399  DOne


In [51]:
k=0
y=0
for i in test_data_classifier2:
    # print(i["argument"])
    # print(i["sentence"], i["start"] , i["end"])
    # print()
    if i["argument"] == i["sentence"][i["start"]: i["end"]]:
        k+=1
    else:
        y+=1
print(k , y)
# all the boundaries of test data classifier2 ,have been correctly identfied

1129 0


In [60]:
k=0
y=0
for i in train_data_classifier2:
    # print(i["argument"])
    # print(i["sentence"], i["start"] , i["end"])
    # print()
    if i["argument"] == i["sentence"][i["start"]: i["end"]]:
        k+=1
    else:
        y+=1
print(k , y)
# all the boundaries of train data classifier2 ,have been correctly identfied

4354 0


In [54]:
test_classifier1_df = pd.DataFrame(test_data_classifier1) 
test_classifier1_df 

,essay no.,sentence,has_argument,type
0,4,International tourism is now more common than ...,False,-
1,4,The last 50 years have seen a significant incr...,False,-
2,4,While some might think the tourism bring larg...,True,Claim
3,4,"Firstly, it is an undeniable fact that tourist...",True,Premise
4,4,"Take Thailand for example, in the Vietnam War...",True,Premise
...,...,...,...,...
1435,398,"Furthermore, it can have a negative affect on ...",True,Premise
1436,398,"For example, a girl, who is interested in lit...",True,Premise
1437,398,"On the other hand, universities should encoura...",True,Premise
1438,398,It would affect students' mental health to st...,True,Premise


In [55]:
train_classifier1_df = pd.DataFrame(train_data_classifier1) 
train_classifier1_df 

,essay no.,sentence,has_argument,type
0,1,Should students be taught to compete or to coo...,False,-
1,1,It is always said that competition can effecti...,False,-
2,1,"In order to survive in the competition, compa...",False,-
3,1,"However, when we discuss the issue of competi...",False,-
4,1,"From this point of view, I firmly believe tha...",True,MajorClaim
...,...,...,...,...
5645,402,"Firstly, children who always playing sport li...",True,Premise
5646,402,"It will be good for children, because indirec...",True,Premise
5647,402,That will make children getting lots of frien...,True,Premise
5648,402,"Secondly, playing sport makes children gettin...",True,Premise


In [56]:
train_classifier2_df = pd.DataFrame(train_data_classifier2) 
train_classifier2_df 

,essay no.,sentence,argument,type,start,end
0,1,"From this point of view, I firmly believe tha...",we should attach more importance to cooperatio...,MajorClaim,48,120
1,1,"First of all, through cooperation, children ca...","through cooperation, children can learn about ...",Claim,14,137
2,1,What we acquired from team work is not only h...,What we acquired from team work is not only ho...,Premise,1,136
3,1,"During the process of cooperation, children c...","During the process of cooperation, children ca...",Premise,1,234
4,1,All of these skills help them to get on well ...,All of these skills help them to get on well w...,Premise,1,104
...,...,...,...,...,...,...
4349,402,"Firstly, children who always playing sport li...",children who always playing sport like footbal...,Premise,10,122
4350,402,"It will be good for children, because indirec...",indirectly they will learn how to socialize ea...,Premise,39,103
4351,402,That will make children getting lots of frien...,they can contribute positively to community,Premise,53,96
4352,402,"Secondly, playing sport makes children gettin...",playing sport makes children getting healthy a...,Premise,11,88


In [57]:
test_classifier2_df = pd.DataFrame(test_data_classifier2) 
test_classifier2_df 

,essay no.,sentence,argument,type,start,end
0,4,While some might think the tourism bring larg...,the tourism bring large profit for the destina...,Claim,24,84
1,4,"Firstly, it is an undeniable fact that tourist...",tourists from different cultures will probably...,Premise,39,152
2,4,"Take Thailand for example, in the Vietnam War...","Take Thailand for example, in the Vietnam War,...",Premise,1,287
3,4,This was due to the lack of adequate controls...,This was due to the lack of adequate controls ...,Premise,1,105
4,4,Therefore this proves that international tour...,international tourism can create negative impa...,Claim,28,106
...,...,...,...,...,...,...
1124,398,"Furthermore, it can have a negative affect on ...",that does not conform to students' personality...,Premise,132,208
1125,398,"For example, a girl, who is interested in lit...",this also can block the girl's future developm...,Premise,148,217
1126,398,"On the other hand, universities should encoura...",this could avoid imbalance of gender in some s...,Premise,130,183
1127,398,It would affect students' mental health to st...,It would affect students' mental health to stu...,Premise,1,85


In [58]:
test_data_file = open("drive/My Drive/AI_LAWYER/test_data_classifier1.csv",'w')
test_classifier1_df.to_csv(test_data_file)
test_data_file.close()
test_data_file = open("drive/My Drive/AI_LAWYER/test_data_classifier2.csv",'w')
test_classifier2_df.to_csv(test_data_file)
test_data_file.close()

In [59]:
train_data_file = open("drive/My Drive/AI_LAWYER/train_data_classifier1.csv",'w')
train_classifier1_df.to_csv(train_data_file)
train_data_file.close()

train_data_file = open("drive/My Drive/AI_LAWYER/train_data_classifier2.csv",'w')
train_classifier2_df.to_csv(train_data_file)
train_data_file.close()
